In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# display all columns
pd.set_option('display.max_columns', None)
# display all rows
pd.set_option('display.max_rows', None)

In [2]:
datacomb_new = pd.read_csv('data_allthreeyears_combined_new1.csv')

/var/folders/5r/_w5lclyd5891k9z55_8lq98r0000gn/T/ipykernel_47402/564117682.py:1: DtypeWarning: Columns (19,24,27,28,30,31,38,39,45,46,48,49,50,52,53,66,67,78,79,82,85,87,110,111,112,115,122,124,133,135,139,140,143,144,145,146,147,148,149,150,151,153,156,157,162,163,164,165,166,167,168,169,170,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  datacomb_new = pd.read_csv('data_allthreeyears_combined_new1.csv')


In [3]:
datacomb_new['Location'].unique()

array(['India', 'Algeria', 'Egypt', 'France', 'Germany', 'Australia', nan,
       'South Korea', 'United States of America', 'Pakistan', 'Mexico',
       'Singapore', 'Italy', 'Taiwan', 'China', 'Argentina', 'Turkey',
       'Brazil', 'Nigeria', 'Chile', 'Japan', 'Canada', 'Indonesia',
       'Israel', 'Colombia', 'Russia',
       'United Kingdom of Great Britain and Northern Ireland', 'Ethiopia',
       'South Africa', 'Viet Nam', 'Kenya', 'Spain',
       'Iran, Islamic Republic of...', 'Cameroon', 'Tunisia', 'Peru',
       'Nepal', 'Morocco', 'Philippines', 'Ecuador', 'Thailand',
       'United Arab Emirates', 'Bangladesh', 'Netherlands', 'Zimbabwe',
       'Ghana', 'Portugal', 'Poland', 'Saudi Arabia', 'Romania',
       'Czech Republic', 'Malaysia', 'Sri Lanka', 'Hong Kong (S.A.R.)',
       'Ukraine', 'Belgium', 'Ireland', 'Greece', 'Switzerland', 'Sweden',
       'Austria', 'Belarus', 'Denmark', 'Uganda', 'Kazakhstan', 'Iraq',
       'Norway', 'Republic of Korea'], dtype=object)

In [4]:
datacomb_new['Location'].nunique()

67

In [5]:
list(datacomb_new.columns)

['Age',
 'Gender - Selected Choice',
 'Location',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Coursera',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - edX',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Kaggle Learn Courses',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - DataCamp',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Fast.ai',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udacity',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udemy',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - LinkedIn Learning',
 'Learning platforms tried - How well known are the platforms (platforms with good 

In [6]:
datacomb_new['year'].dtype

dtype('int64')

In [7]:
datacomb_new.shape

(70006, 243)

In [8]:
categorical_columns = datacomb_new.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['Age',
 'Gender - Selected Choice',
 'Location',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Coursera',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - edX',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Kaggle Learn Courses',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - DataCamp',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Fast.ai',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udacity',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udemy',
 'Learning platforms tried - How well known are the platforms (platforms with good marketing) - LinkedIn Learning',
 'Learning platforms tried - How well known are the platforms (platforms with good 

# drop the 'year' column

In [9]:
# drop the 'year' column

datacomb_new = datacomb_new.drop('year', axis = 1)

In [10]:
datacomb_new.shape

(70006, 242)

In [11]:
unique_counts = datacomb_new.nunique(dropna=False)

In [12]:
# unique_counts

In [13]:
binary_cols = unique_counts[unique_counts <= 2].index.tolist()

In [14]:
non_binary_cols = unique_counts[unique_counts > 2].index.tolist()
non_binary_cols

['Age',
 'Gender - Selected Choice',
 'Location',
 'Education level_attainedOrGGtoAttain',
 'Coding Experience (in years)',
 'Years in ML',
 'Job_title - Selected Choice',
 'Job_No.OfDSTeamMember',
 'Job_EmployerUsingML?',
 'Job_Salary',
 'Money Spent on ML/Cloud Computing',
 'Times used TPU']

In [15]:
# number of binary cols in the dataset
len(binary_cols)

230

In [16]:
datacomb_new.shape

(70006, 242)

In [17]:
datacomb_new.sample(2)

,Age,Gender - Selected Choice,Location,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Coursera,Learning platforms tried - How well known are the platforms (platforms with good marketing) - edX,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Kaggle Learn Courses,Learning platforms tried - How well known are the platforms (platforms with good marketing) - DataCamp,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Fast.ai,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udacity,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udemy,Learning platforms tried - How well known are the platforms (platforms with good marketing) - LinkedIn Learning,"Learning platforms tried - How well known are the platforms (platforms with good marketing) - Cloud-certification programs (direct from AWS, Azure, GCP, or similar)",Learning platforms tried - How well known are the platforms (platforms with good marketing) - University Courses (resulting in a university degree),Education level_attainedOrGGtoAttain,Coding Experience (in years),Popular programming language - Python,Popular programming language - R,Popular programming language - SQL,Popular programming language - C,Popular programming language - C#,Popular programming language - C++,Popular programming language - Java,Popular programming language - Javascript,Popular programming language - Bash,Popular programming language - PHP,Popular programming language - MATLAB,Popular programming language - Julia,Popular programming language - Go,Popular IDEs - JupyterLab,Popular IDEs - RStudio,Popular IDEs - Visual Studio,Popular IDEs - Visual Studio Code (VSCode),Popular IDEs - PyCharm,Popular IDEs - Spyder,Popular IDEs - Notepad++,Popular IDEs - Sublime Text,Popular IDEs - Vim / Emacs,Popular IDEs - MATLAB,Popular IDEs - Jupyter Notebook,Popular IDEs - IntelliJ,Popular hosted notebook products - Kaggle Notebooks,Popular hosted notebook products - Colab Notebooks,Popular hosted notebook products - Azure Notebooks,Popular hosted notebook products - Code Ocean,Popular hosted notebook products - IBM Watson Studio,Popular hosted notebook products - Amazon Sagemaker Studio,Popular hosted notebook products - Amazon Sagemaker Studio Lab,Popular hosted notebook products - Amazon EMR Notebooks,Popular hosted notebook products - Google Cloud Vertex AI Workbench,Popular hosted notebook products - Hex Workspaces,Popular hosted notebook products - Noteable Notebooks,Popular hosted notebook products - Databricks Collaborative Notebooks,Popular hosted notebook products - Deepnote Notebooks,Popular hosted notebook products - Gradient Notebooks,Years in ML,Popular ML frameworks - Scikit-learn,Popular ML frameworks - TensorFlow,Popular ML frameworks - Keras,Popular ML frameworks - PyTorch,Popular ML frameworks - Fast.ai,Popular ML frameworks - Xgboost,Popular ML frameworks - LightGBM,Popular ML frameworks - CatBoost,Popular ML frameworks - Caret,Popular ML frameworks - Tidymodels,Popular ML frameworks - JAX,Popular ML frameworks - PyTorch Lightning,Popular ML frameworks - Huggingface,Popular ML Algorithms - Linear or Logistic Regression,Popular ML Algorithms - Decision Trees or Random Forests,"Popular ML Algorithms - Gradient Boosting Machines (xgboost, lightgbm, etc)",Popular ML Algorithms - Bayesian Approaches,Popular ML Algorithms - Evolutionary Approaches,"Popular ML Algorithms - Dense Neural Networks (MLPs, etc)",Popular ML Algorithms - Convolutional Neural Networks,Popular ML Algorithms - Generative Adversarial Networks,Popular ML Algorithms - Recurrent Neural Networks,"Popular ML Algorithms - Transformer Networks (BERT, gpt-3, etc)","Popular ML Algorithms - Autoencoder Networks (DAE, VAE, etc)",Popular ML Algorithms - Graph Neural Networks,"Popular Computer Vision Methods - General purpose 

In [18]:
# binary_cols

In [19]:
# # sanity check only
# for i in binary_cols:
#     print(i)
#     print(datacomb_new[i].unique())
#     print('=====================')
    

In [20]:
# # sanity check only
# for i in non_binary_cols:
#     print(i)
#     print(datacomb_new[i].unique())
#     print('=====================')
    

In [21]:
non_binary_cols

['Age',
 'Gender - Selected Choice',
 'Location',
 'Education level_attainedOrGGtoAttain',
 'Coding Experience (in years)',
 'Years in ML',
 'Job_title - Selected Choice',
 'Job_No.OfDSTeamMember',
 'Job_EmployerUsingML?',
 'Job_Salary',
 'Money Spent on ML/Cloud Computing',
 'Times used TPU']

In [22]:
# check if the columns in non_binary_cols have NaN values
# columns with no NaN values may have the multi-colinearity problem when we one-hot-encode it with pd.get_dummies
for i in non_binary_cols:
    print(i, "---------", datacomb_new[i].isna().any().any())

Age --------- False
Gender - Selected Choice --------- True
Location --------- True
Education level_attainedOrGGtoAttain --------- True
Coding Experience (in years) --------- True
Years in ML --------- True
Job_title - Selected Choice --------- True
Job_No.OfDSTeamMember --------- True
Job_EmployerUsingML? --------- True
Job_Salary --------- True
Money Spent on ML/Cloud Computing --------- True
Times used TPU --------- True


In [23]:
datacomb_new[non_binary_cols].count()

Age                                     70006
Gender - Selected Choice                68707
Location                                65807
Education level_attainedOrGGtoAttain    66520
Coding Experience (in years)            68336
Years in ML                             60014
Job_title - Selected Choice             55880
Job_No.OfDSTeamMember                   36398
Job_EmployerUsingML?                    35954
Job_Salary                              34256
Money Spent on ML/Cloud Computing       34237
Times used TPU                          45776
dtype: int64

# Label Binary Columns to 0 and 1

In [24]:
datacomb_new[binary_cols] = np.where((datacomb_new[binary_cols] != 0) & (~datacomb_new[binary_cols].isna()), 1, 0)

In [25]:
datacomb_new.shape

(70006, 242)

In [26]:
datacomb_new['Job_title - Selected Choice'].isna().sum() # number of rows with the Job_title blank

14126

In [27]:
# drop rows with empty job title
datacomb_new = datacomb_new.dropna(subset = ['Job_title - Selected Choice'])

In [28]:
datacomb_new['Job_title - Selected Choice'].isna().sum() # number of rows with the Job_title blank is now 0

0

In [29]:
datacomb_new.shape

(55880, 242)

In [30]:
# drop rows with student as job title
datacomb_new = datacomb_new[datacomb_new['Job_title - Selected Choice'] != 'Student']
datacomb_new.shape

(43905, 242)

In [31]:
Job_title = datacomb_new.pop('Job_title - Selected Choice')
datacomb_new.insert(len(datacomb_new.columns), 'Job_title - Selected Choice', Job_title)

/var/folders/5r/_w5lclyd5891k9z55_8lq98r0000gn/T/ipykernel_47402/2434528012.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datacomb_new.insert(len(datacomb_new.columns), 'Job_title - Selected Choice', Job_title)


In [32]:
datacomb_new.head(1)

,Age,Gender - Selected Choice,Location,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Coursera,Learning platforms tried - How well known are the platforms (platforms with good marketing) - edX,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Kaggle Learn Courses,Learning platforms tried - How well known are the platforms (platforms with good marketing) - DataCamp,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Fast.ai,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udacity,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udemy,Learning platforms tried - How well known are the platforms (platforms with good marketing) - LinkedIn Learning,"Learning platforms tried - How well known are the platforms (platforms with good marketing) - Cloud-certification programs (direct from AWS, Azure, GCP, or similar)",Learning platforms tried - How well known are the platforms (platforms with good marketing) - University Courses (resulting in a university degree),Education level_attainedOrGGtoAttain,Coding Experience (in years),Popular programming language - Python,Popular programming language - R,Popular programming language - SQL,Popular programming language - C,Popular programming language - C#,Popular programming language - C++,Popular programming language - Java,Popular programming language - Javascript,Popular programming language - Bash,Popular programming language - PHP,Popular programming language - MATLAB,Popular programming language - Julia,Popular programming language - Go,Popular IDEs - JupyterLab,Popular IDEs - RStudio,Popular IDEs - Visual Studio,Popular IDEs - Visual Studio Code (VSCode),Popular IDEs - PyCharm,Popular IDEs - Spyder,Popular IDEs - Notepad++,Popular IDEs - Sublime Text,Popular IDEs - Vim / Emacs,Popular IDEs - MATLAB,Popular IDEs - Jupyter Notebook,Popular IDEs - IntelliJ,Popular hosted notebook products - Kaggle Notebooks,Popular hosted notebook products - Colab Notebooks,Popular hosted notebook products - Azure Notebooks,Popular hosted notebook products - Code Ocean,Popular hosted notebook products - IBM Watson Studio,Popular hosted notebook products - Amazon Sagemaker Studio,Popular hosted notebook products - Amazon Sagemaker Studio Lab,Popular hosted notebook products - Amazon EMR Notebooks,Popular hosted notebook products - Google Cloud Vertex AI Workbench,Popular hosted notebook products - Hex Workspaces,Popular hosted notebook products - Noteable Notebooks,Popular hosted notebook products - Databricks Collaborative Notebooks,Popular hosted notebook products - Deepnote Notebooks,Popular hosted notebook products - Gradient Notebooks,Years in ML,Popular ML frameworks - Scikit-learn,Popular ML frameworks - TensorFlow,Popular ML frameworks - Keras,Popular ML frameworks - PyTorch,Popular ML frameworks - Fast.ai,Popular ML frameworks - Xgboost,Popular ML frameworks - LightGBM,Popular ML frameworks - CatBoost,Popular ML frameworks - Caret,Popular ML frameworks - Tidymodels,Popular ML frameworks - JAX,Popular ML frameworks - PyTorch Lightning,Popular ML frameworks - Huggingface,Popular ML Algorithms - Linear or Logistic Regression,Popular ML Algorithms - Decision Trees or Random Forests,"Popular ML Algorithms - Gradient Boosting Machines (xgboost, lightgbm, etc)",Popular ML Algorithms - Bayesian Approaches,Popular ML Algorithms - Evolutionary Approaches,"Popular ML Algorithms - Dense Neural Networks (MLPs, etc)",Popular ML Algorithms - Convolutional Neural Networks,Popular ML Algorithms - Generative Adversarial Networks,Popular ML Algorithms - Recurrent Neural Networks,"Popular ML Algorithms - Transformer Networks (BERT, gpt-3, etc)","Popular ML Algorithms - Autoencoder Networks (DAE, VAE, etc)",Popular ML Algorithms - Graph Neural Networks,"Popular Computer Vision Methods - General purpose 

In [33]:
print(len(binary_cols))
print(len(non_binary_cols))

230
12


# Dropping cols we think is not associated to the job title

In [109]:
datacomb_new.shape

(43905, 242)

In [142]:
job_title_dict = {
    'Data Analyst (Business, Marketing, Financial, Quantitative, etc)': 'Data Analyst',
    'Product Manager': 'Product/Project/Program Manager',
    'Product/Project Manager': 'Product/Project/Program Manager',
    'Program/Project Manager':'Product/Project/Program Manager',
    'Machine Learning Engineer':'Machine Learning/ MLops Engineer'}

def replace_text(cell_value, replacements):
    if cell_value is not None and not pd.isna(cell_value):
        # Check if the cell_value is a float, and if so, convert it to a string.
        if isinstance(cell_value, float):
            cell_value = str(cell_value)
        cell_value = replacements.get(cell_value,cell_value)
    return cell_value

datacomb_new['Job_title - Selected Choice'] = datacomb_new['Job_title - Selected Choice'].apply(replace_text, replacements=job_title_dict)

In [143]:
Job_title = datacomb_new.pop('Job_title - Selected Choice')

In [134]:
cols_to_drop = ['Job_No.OfDSTeamMember', 'Job_EmployerUsingML?','Money Spent on ML/Cloud Computing','Times used TPU', 'Job_title - Selected Choice']
datacomb_new_wo_Jtitle = datacomb_new.drop(cols_to_drop, axis = 1)

In [135]:
filtered_non_binary_cols = [item for item in non_binary_cols if item not in cols_to_drop]

In [136]:
filtered_non_binary_cols

['Age',
 'Gender - Selected Choice',
 'Location',
 'Education level_attainedOrGGtoAttain',
 'Coding Experience (in years)',
 'Years in ML',
 'Job_Salary']

In [38]:
 # = datacomb_new.drop('Job_title - Selected Choice', axis = 1)

In [137]:
encoded_df = pd.get_dummies(datacomb_new_wo_Jtitle, columns = filtered_non_binary_cols)

In [138]:
encoded_df.shape

(43905, 346)

In [139]:
encoded_df[encoded_df['Popular IDEs - Click to write Choice 13'] ==1].count()

Learning platforms tried - How well known are the platforms (platforms with good marketing) - Coursera                                                                  4190
Learning platforms tried - How well known are the platforms (platforms with good marketing) - edX                                                                       4190
Learning platforms tried - How well known are the platforms (platforms with good marketing) - Kaggle Learn Courses                                                      4190
Learning platforms tried - How well known are the platforms (platforms with good marketing) - DataCamp                                                                  4190
Learning platforms tried - How well known are the platforms (platforms with good marketing) - Fast.ai                                                                   4190
Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udacity                                  

In [42]:
encoded_df.drop('Age_70+', axis = 1, inplace = True) # to remove multi-colinearity

In [43]:
encoded_df.shape # verifying that there is one less column

(43905, 345)

In [44]:
encoded_df.head()

,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Coursera,Learning platforms tried - How well known are the platforms (platforms with good marketing) - edX,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Kaggle Learn Courses,Learning platforms tried - How well known are the platforms (platforms with good marketing) - DataCamp,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Fast.ai,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udacity,Learning platforms tried - How well known are the platforms (platforms with good marketing) - Udemy,Learning platforms tried - How well known are the platforms (platforms with good marketing) - LinkedIn Learning,"Learning platforms tried - How well known are the platforms (platforms with good marketing) - Cloud-certification programs (direct from AWS, Azure, GCP, or similar)",Learning platforms tried - How well known are the platforms (platforms with good marketing) - University Courses (resulting in a university degree),Popular programming language - Python,Popular programming language - R,Popular programming language - SQL,Popular programming language - C,Popular programming language - C#,Popular programming language - C++,Popular programming language - Java,Popular programming language - Javascript,Popular programming language - Bash,Popular programming language - PHP,Popular programming language - MATLAB,Popular programming language - Julia,Popular programming language - Go,Popular IDEs - JupyterLab,Popular IDEs - RStudio,Popular IDEs - Visual Studio,Popular IDEs - Visual Studio Code (VSCode),Popular IDEs - PyCharm,Popular IDEs - Spyder,Popular IDEs - Notepad++,Popular IDEs - Sublime Text,Popular IDEs - Vim / Emacs,Popular IDEs - MATLAB,Popular IDEs - Jupyter Notebook,Popular IDEs - IntelliJ,Popular hosted notebook products - Kaggle Notebooks,Popular hosted notebook products - Colab Notebooks,Popular hosted notebook products - Azure Notebooks,Popular hosted notebook products - Code Ocean,Popular hosted notebook products - IBM Watson Studio,Popular hosted notebook products - Amazon Sagemaker Studio,Popular hosted notebook products - Amazon Sagemaker Studio Lab,Popular hosted notebook products - Amazon EMR Notebooks,Popular hosted notebook products - Google Cloud Vertex AI Workbench,Popular hosted notebook products - Hex Workspaces,Popular hosted notebook products - Noteable Notebooks,Popular hosted notebook products - Databricks Collaborative Notebooks,Popular hosted notebook products - Deepnote Notebooks,Popular hosted notebook products - Gradient Notebooks,Popular ML frameworks - Scikit-learn,Popular ML frameworks - TensorFlow,Popular ML frameworks - Keras,Popular ML frameworks - PyTorch,Popular ML frameworks - Fast.ai,Popular ML frameworks - Xgboost,Popular ML frameworks - LightGBM,Popular ML frameworks - CatBoost,Popular ML frameworks - Caret,Popular ML frameworks - Tidymodels,Popular ML frameworks - JAX,Popular ML frameworks - PyTorch Lightning,Popular ML frameworks - Huggingface,Popular ML Algorithms - Linear or Logistic Regression,Popular ML Algorithms - Decision Trees or Random Forests,"Popular ML Algorithms - Gradient Boosting Machines (xgboost, lightgbm, etc)",Popular ML Algorithms - Bayesian Approaches,Popular ML Algorithms - Evolutionary Approaches,"Popular ML Algorithms - Dense Neural Networks (MLPs, etc)",Popular ML Algorithms - Convolutional Neural Networks,Popular ML Algorithms - Generative Adversarial Networks,Popular ML Algorithms - Recurrent Neural Networks,"Popular ML Algorithms - Transformer Networks (BERT, gpt-3, etc)","Popular ML Algorithms - Autoencoder Networks (DAE, VAE, etc)",Popular ML Algorithms - Graph Neural Networks,"Popular Computer Vision Methods - General purpose image/video tools (PIL, cv2, skimage, etc)","Popular Computer Vision Methods - Image segmentation methods (U-Net, Ma

In [94]:
a = list(encoded_df.columns)
a.sort()
a

['Age_18-21',
 'Age_22-24',
 'Age_25-29',
 'Age_30-34',
 'Age_35-39',
 'Age_40-44',
 'Age_45-49',
 'Age_50-54',
 'Age_55-59',
 'Age_60-69',
 'Coding Experience (in years)_1-2 years',
 'Coding Experience (in years)_1-3 years',
 'Coding Experience (in years)_10-20 years',
 'Coding Experience (in years)_20+ years',
 'Coding Experience (in years)_3-5 years',
 'Coding Experience (in years)_5-10 years',
 'Coding Experience (in years)_< 1 years',
 'Coding Experience (in years)_I have never written code',
 'Education level_attainedOrGGtoAttain_bachelors',
 'Education level_attainedOrGGtoAttain_college without bachelors',
 'Education level_attainedOrGGtoAttain_doctoral',
 'Education level_attainedOrGGtoAttain_doctorate',
 'Education level_attainedOrGGtoAttain_high school and below',
 'Education level_attainedOrGGtoAttain_masters',
 'Education level_attainedOrGGtoAttain_professional deg',
 'Gender - Selected Choice_Man',
 'Gender - Selected Choice_Woman',
 'Job_JobScope - Analyze and understand 

In [46]:
# encoded_array = encoded_df.to_numpy()

In [47]:
# encoded_array

array([[1, 0, 1, ..., False, False, False],
       [1, 1, 0, ..., False, False, False],
       [1, 0, 1, ..., False, False, False],
       ...,
       [0, 0, 0, ..., False, False, False],
       [1, 0, 0, ..., False, False, False],
       [1, 1, 1, ..., False, False, False]], dtype=object)

In [48]:
# len(encoded_array[1])

345

In [49]:
# encoded_array[1].reshape(1,-1)

array([[1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, False, False, False, True, False,
        False, False, False, False, False, True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, Fals

In [50]:
from sklearn.model_selection import train_test_split

rng = np.random.RandomState(seed=321)
X_train, X_test, y_train, y_test = train_test_split( encoded_df, Job_title , test_size=0.20, random_state= rng)

print( X_train.shape )
print( X_test.shape )

(35124, 345)
(8781, 345)


In [51]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier( n_estimators=100, max_leaf_nodes=15, n_jobs=-1 )

rnd_clf.fit( X_train, y_train )

y_pred_rf = rnd_clf.predict( X_test )

In [52]:
from sklearn.metrics import classification_report

# print( "Decision Tree")
# print( classification_report( y_test, y_pred_dt ))
print( "Random Forest")
print( classification_report( y_test, y_pred_rf ))

Random Forest
                                                                  precision    recall  f1-score   support

                                                Business Analyst       0.00      0.00      0.00       369
                                          Currently not employed       0.40      0.96      0.57       990
                                           DBA/Database Engineer       0.00      0.00      0.00        51
                                              Data Administrator       0.00      0.00      0.00        17
                                                    Data Analyst       0.32      0.04      0.08       769
Data Analyst (Business, Marketing, Financial, Quantitative, etc)       0.00      0.00      0.00       307
                                                  Data Architect       0.00      0.00      0.00        22
                                                   Data Engineer       0.00      0.00      0.00       288
                               

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [53]:
# finding best n and depth

from sklearn.model_selection import GridSearchCV

params_to_test = {
    'n_estimators':[10,25,50],#,100,200,500
    'max_depth':[3,5,10] #15,20,25
}

#here you can put any parameter you want at every run, like random_state or verbosity
rf_model = RandomForestClassifier(random_state=rng)
#here you specify the CV parameters, number of folds, numberof cores to use...
grid_search = GridSearchCV(rf_model, param_grid=params_to_test, cv=10, scoring='f1_macro', n_jobs=4)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_ 

#best_params is a dict you can pass directly to train a model with optimal settings 
best_model = RandomForestClassifier(**best_params)

In [54]:
best_model

RandomForestClassifier(max_depth=10, n_estimators=25)

In [58]:
best_model.fit( X_train, y_train )
best_model

RandomForestClassifier(max_depth=10, n_estimators=25)

In [59]:
y_pred_rf = best_model.predict( X_test )

In [60]:

print( classification_report( y_test, y_pred_rf ))

                                                                  precision    recall  f1-score   support

                                                Business Analyst       0.50      0.00      0.01       369
                                          Currently not employed       0.41      0.94      0.58       990
                                           DBA/Database Engineer       0.00      0.00      0.00        51
                                              Data Administrator       0.00      0.00      0.00        17
                                                    Data Analyst       0.31      0.17      0.22       769
Data Analyst (Business, Marketing, Financial, Quantitative, etc)       0.75      0.01      0.02       307
                                                  Data Architect       0.00      0.00      0.00        22
                                                   Data Engineer       0.00      0.00      0.00       288
                                             

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [66]:
# Finding best model (2)
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [69]:
# # !!! resource intensive !!!
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestClassifier()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=  50.8s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time= 1.4min
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time= 1.5min


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=  36.9s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=  52.9s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time= 1.6min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=800; total time= 1.9min
[CV] END bootstrap=True, max_depth=10, 

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
123 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
41 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/base.py", line 638, in _valid

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [118]:
rf_random.best_params_
# {'n_estimators': 1600,
#  'min_samples_split': 5,
#  'min_samples_leaf': 1,
#  'max_features': 'sqrt',
#  'max_depth': 70,
#  'bootstrap': False}

{'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': False}

In [76]:
rf_random.best_score_ 


0.4311866530007972

In [83]:
best_model_2 = RandomForestClassifier(**rf_random.best_params_)
best_model_2

RandomForestClassifier(bootstrap=False, max_depth=70, min_samples_split=5,
                       n_estimators=1600)

In [84]:
best_model_2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=70, min_samples_split=5,
                       n_estimators=1600)

In [87]:
y_pred_best_2 = best_model_2.predict(X_test)

In [89]:
print(classification_report( y_test, y_pred_best_2 ))

# remove
# 'Currently not employed'

# merge

                                                                  precision    recall  f1-score   support

                                                Business Analyst       0.15      0.02      0.03       369
                                          Currently not employed       0.63      0.92      0.75       990
                                           DBA/Database Engineer       1.00      0.04      0.08        51
                                              Data Administrator       0.00      0.00      0.00        17
                                                    Data Analyst       0.32      0.30      0.31       769
Data Analyst (Business, Marketing, Financial, Quantitative, etc)       0.42      0.10      0.16       307
                                                  Data Architect       0.00      0.00      0.00        22
                                                   Data Engineer       0.23      0.01      0.02       288
                                             

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [90]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Calculate precision
precision = precision_score(y_test, y_pred_best_2, average='micro')

# Calculate recall
recall = recall_score(y_test, y_pred_best_2, average='micro')


print("Precision: ", precision)
print("Recall: ", recall)


Precision:  0.42842500854116844
Recall:  0.42842500854116844


In [ ]:
# from sklearn.model_selection import GridSearchCV
# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [80, 90, 100, 110],
#     'max_features': [2, 3],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [100, 200, 300, 1000]
# }
# # Create a based model
# rf = RandomForestClassifier()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)

## Exploration 1
1. Remove country

In [140]:
exploration_df_1 = datacomb_new_wo_Jtitle.copy()
exploration_df_1 = exploration_df_1[exploration_df_1.columns.drop(list(exploration_df_1.filter(regex='Location')))]

pprint(list(exploration_df_1.columns))
exploration_df_1.shape

['Age',
 'Gender - Selected Choice',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - Coursera',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - edX',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - Kaggle Learn Courses',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - DataCamp',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - Fast.ai',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - Udacity',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - Udemy',
 'Learning platforms tried - How well known are the platforms (platforms with '
 'good marketing) - LinkedIn Learning',
 'Learning platforms tried - How well known are the platforms (p

(43905, 236)

KeyError: 'Job_Title'

In [114]:
filtered_non_binary_cols_dum = filtered_non_binary_cols.copy()
exploration_1_encoded_df = pd.get_dummies(exploration_df_1, columns = filtered_non_binary_cols_dum.remove('Location'))
exploration_1_encoded_df.shape

(43905, 279)

In [116]:
exploration_1_encoded_df.drop('Age_70+', axis = 1, inplace = True) # to remove multi-colinearity

In [117]:
a = list(exploration_1_encoded_df.columns)
a.sort()
a

['Age_18-21',
 'Age_22-24',
 'Age_25-29',
 'Age_30-34',
 'Age_35-39',
 'Age_40-44',
 'Age_45-49',
 'Age_50-54',
 'Age_55-59',
 'Age_60-69',
 'Coding Experience (in years)_1-2 years',
 'Coding Experience (in years)_1-3 years',
 'Coding Experience (in years)_10-20 years',
 'Coding Experience (in years)_20+ years',
 'Coding Experience (in years)_3-5 years',
 'Coding Experience (in years)_5-10 years',
 'Coding Experience (in years)_< 1 years',
 'Coding Experience (in years)_I have never written code',
 'Education level_attainedOrGGtoAttain_bachelors',
 'Education level_attainedOrGGtoAttain_college without bachelors',
 'Education level_attainedOrGGtoAttain_doctoral',
 'Education level_attainedOrGGtoAttain_doctorate',
 'Education level_attainedOrGGtoAttain_high school and below',
 'Education level_attainedOrGGtoAttain_masters',
 'Education level_attainedOrGGtoAttain_professional deg',
 'Gender - Selected Choice_Man',
 'Gender - Selected Choice_Woman',
 'Job_JobScope - Analyze and understand 

In [144]:
# train test split
X_train, X_test, y_train, y_test = train_test_split( exploration_1_encoded_df, Job_title , test_size=0.20, random_state= rng)

print( X_train.shape )
print( X_test.shape )

(35124, 278)
(8781, 278)


In [120]:

exploration_1_rnd_clf = RandomForestClassifier(**rf_random.best_params_)

exploration_1_rnd_clf.fit( X_train, y_train )

exploration_1_y_pred_rf = exploration_1_rnd_clf.predict( X_test )

In [149]:
def evaluate(y_test, y_pred):
    print(classification_report( y_test, y_pred ))
    # Calculate precision
    precision = precision_score(y_test, y_pred, average='micro')
    
    # Calculate recall
    recall = recall_score(y_test, y_pred, average='micro')
    
    print("Precision: ", precision)
    print("Recall: ", recall)


In [122]:
# Calculate precision
precision = precision_score(y_test, exploration_1_y_pred_rf, average='micro')

# Calculate recall
recall = recall_score(y_test, exploration_1_y_pred_rf, average='micro')


print("Precision: ", precision)
print("Recall: ", recall)

Precision:  0.43058877121056827
Recall:  0.43058877121056827


## Exploration 2
1. Remove country
2. Remove IDE

In [ ]:
# Popular IDEs -

## "Final"

In [126]:
hyper_parameters = {'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': False}

In [172]:
jobs_df = pd.DataFrame(y_train)
jobs_count = pd.DataFrame(jobs_df.groupby('Job_title - Selected Choice')['Job_title - Selected Choice'].count())
jobs_count.reset_index(drop = True, inplace = True)
jobs_count.sort_values('Job_title - Selected Choice')

# jobs_df.groupby('Job_title - Selected Choice').count()

,Job_title - Selected Choice
3,50
8,51
5,76
9,78
2,237
10,381
17,590
12,658
18,681
6,1142


In [176]:
final_rf = RandomForestClassifier(**hyper_parameters)
final_rf

RandomForestClassifier(bootstrap=False, max_depth=70, min_samples_split=5,
                       n_estimators=1600)

In [182]:
final_rf.n_outputs_ = 3

In [ ]:
final_rf.fit(X_train, y_train)

In [ ]:
# final_rf.n_outputs_ = 3
final_y_pred = final_rf.predict(X_test)
final_y_pred

In [150]:
evaluate(y_test, final_y_pred)

                                                              precision    recall  f1-score   support

                                            Business Analyst       0.22      0.02      0.03       371
                                      Currently not employed       0.63      0.92      0.75       995
                                       DBA/Database Engineer       1.00      0.02      0.03        59
                                          Data Administrator       0.00      0.00      0.00        20
                                                Data Analyst       0.36      0.44      0.39      1052
                                              Data Architect       0.00      0.00      0.00        19
                                               Data Engineer       0.60      0.02      0.04       315
                                              Data Scientist       0.40      0.69      0.51      1622
                                          Developer Advocate       0.00      0.00

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [152]:
a= list(exploration_1_encoded_df.columns)
a.sort()
a

['Age_18-21',
 'Age_22-24',
 'Age_25-29',
 'Age_30-34',
 'Age_35-39',
 'Age_40-44',
 'Age_45-49',
 'Age_50-54',
 'Age_55-59',
 'Age_60-69',
 'Coding Experience (in years)_1-2 years',
 'Coding Experience (in years)_1-3 years',
 'Coding Experience (in years)_10-20 years',
 'Coding Experience (in years)_20+ years',
 'Coding Experience (in years)_3-5 years',
 'Coding Experience (in years)_5-10 years',
 'Coding Experience (in years)_< 1 years',
 'Coding Experience (in years)_I have never written code',
 'Education level_attainedOrGGtoAttain_bachelors',
 'Education level_attainedOrGGtoAttain_college without bachelors',
 'Education level_attainedOrGGtoAttain_doctoral',
 'Education level_attainedOrGGtoAttain_doctorate',
 'Education level_attainedOrGGtoAttain_high school and below',
 'Education level_attainedOrGGtoAttain_masters',
 'Education level_attainedOrGGtoAttain_professional deg',
 'Gender - Selected Choice_Man',
 'Gender - Selected Choice_Woman',
 'Job_JobScope - Analyze and understand 

In [173]:
from sklearn.tree import export_graphviz

export_graphviz(final_rf,
                feature_names=exploration_1_encoded_df.columns,
                filled=True,
                rounded=True)

os.system('dot -Tpng tree.dot -o tree.png')


AttributeError: 'RandomForestClassifier' object has no attribute 'tree_'